In [ ]:
"""

Compute the average T and S per ice shelf to check if there is some link to non-viability date

"""

In [ ]:
import xarray as xr
from tqdm.notebook import tqdm

import summer_paper.data_formatting_NN as dfmt
import summer_paper.useful_functions as uf
import multimelt.melt_functions as meltf


In [ ]:
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
ano_choice_file = xr.open_dataset(inputpath_weights + 'ano_choice_NEMOorISMIP_withoutGISS.nc')

In [ ]:
home_path = '/bettik/burgardc/'

inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'
inputpath_CVinput = home_path+'/DATA/NN_PARAM/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/'
inputpath_plumes = home_path+'/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'
inputpath_boxes = home_path+'/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_BedMachine = dfmt.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_bed_goodGL = -1*file_BedMachine['bed']
file_draft = (file_BedMachine['thickness'] - file_BedMachine['surface']).where(file_isf['ISF_mask'] > 1)
file_isf_conc = file_BedMachine['isf_conc']

grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
cell_area_weight = true_grid_cell_area/(4000 * 4000)

lon = file_isf.longitude
lat = file_isf.latitude

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area_const = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

ice_draft_pos = file_draft
ice_draft_neg = -ice_draft_pos

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos.rename('ice_draft_pos')).merge(grid_cell_area_weighted.rename('grid_cell_area_weighted')).merge(file_isf_conc.rename('isfdraft_conc'))
geometry_info_1D = param_var_of_int_1D

norm_metrics_file = xr.open_dataset(inputpath_CVinput + 'metrics_norm_wholedataset_origexcept26_christoph_new.nc')
norm_metrics = norm_metrics_file.to_dataframe()

box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=1).drop('Nisf')

file_slope = xr.open_dataset(inputpath_mask+'BedMachine_4km_slope_info_bedrock_draft_latlon_oneFRIS.nc')

In [ ]:
ano_choice_file['ano_choice'].load()

In [ ]:
file_TS_0

In [ ]:
filled_TS['theta_ocean'].depth

In [ ]:
depth_of_int

In [ ]:
filled_TS['theta_ocean'].sel(Nisf=kisf).interp({'depth': depth_of_int})

In [ ]:
filled_TS['theta_ocean'].sel(Nisf=kisf)

In [ ]:
T_S_2D_meanstd_kisf

In [ ]:
### APPLY MODEL
scenario = 'ssp245'

### if we want the "real" profiles and not ISMIP + anomalies
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'
T_ISMIP = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_ISMIP.nc')
S_ISMIP = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_ISMIP.nc')
TS_ISMIP = xr.merge([T_ISMIP.rename({'thetao':'theta_ocean'}),S_ISMIP.rename({'so':'salinity_ocean'})])
    
inputpath_profiles_NEMO = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_OPM026/'
file_TS_orig = xr.open_dataset(inputpath_profiles_NEMO+'T_S_mean_prof_corrected_km_contshelf_1980-2018_rignotisf.nc')
TS_NEMO = file_TS_orig.sel(profile_domain=[50]).squeeze().drop('profile_domain').mean('time')
TS_NEMO['depth'] = -1*TS_NEMO['depth']
TS_NEMO_rightaxis = TS_NEMO.rename({'depth':'z'}).interp({'z': TS_ISMIP.z})
TS_NEMO_withISMIP = xr.concat([TS_NEMO_rightaxis,TS_ISMIP.sel(Nisf=[36,62])], dim='Nisf')


### If no ensemble of models
for mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
           'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','IPSL-CM6A-LR','MPI-ESM1-2-HR',
           'MRI-ESM2-0','UKESM1-0-LL']: #
#for mod in []: #,,
    print(mod)
    
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        to2300 = True     
    elif mod == 'CESM2':
        to2300 = False        

    if scenario == 'historical':
        yystart = 1850 #1980 #1850
        yyend = 2014
    elif scenario == 'ssp245':
        yystart = 2015
        yyend = 2100  
    else:
        if to2300:
            yystart = 2015
            yyend = 2300
        else:
            yystart = 2015
            yyend = 2100 

    inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

    T_Clim = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    S_Clim = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    TS_Clim = xr.merge([T_Clim.rename({'thetao':'theta_ocean'}),S_Clim.rename({'so':'salinity_ocean'})])
            
    # PREPARE VARIABLES
    file_TS_list = []
    for tt in range(yystart,yyend+1):
        file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
        file_TS_list.append(file_TS_orig)

    file_TS_0 = xr.concat(file_TS_list, dim='time')
    
    file_TS_NEMO = (file_TS_0 - TS_ISMIP + TS_NEMO_withISMIP).rename({'z':'depth'})
    file_TS = file_TS_0.rename({'z':'depth'}).where(ano_choice_file['ano_choice'] != 'NEMO', file_TS_NEMO)

    file_TS['depth'] = -1*file_TS['depth']
    depth_axis_old = file_TS.depth.values
    depth_axis_new = np.concatenate((np.zeros(1),depth_axis_old))
    file_TS_with_shallow = file_TS.interp({'depth': depth_axis_new})
    filled_TS = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth').sel(Nisf=file_isf.Nisf) #, 'profile_domain': 1})
    

    all_info_list = []
    
    print('Prepare input variables')
    n = 0
    for kisf in tqdm(file_isf.Nisf):
        depth_kisf = uf.choose_isf(ice_draft_pos,isf_stack_mask, kisf)
        depth_of_int0 = depth_kisf.where(depth_kisf < file_isf['front_bot_depth_max'].sel(Nisf=kisf), 
                                       file_isf['front_bot_depth_max'].sel(Nisf=kisf))
        depth_of_int = depth_of_int0.where(depth_kisf > file_isf['front_ice_depth_min'].sel(Nisf=kisf), 
                                       file_isf['front_ice_depth_min'].sel(Nisf=kisf))

        T_isf = filled_TS['theta_ocean'].sel(Nisf=kisf).interp({'depth': depth_of_int}).drop('depth')
        S_isf = filled_TS['salinity_ocean'].sel(Nisf=kisf).interp({'depth': depth_of_int}).drop('depth')

        cell_area_kisf = uf.choose_isf(cell_area_weight,isf_stack_mask, kisf) 
        isf_conc_kisf = uf.choose_isf(file_isf_conc,isf_stack_mask, kisf) 
        weight_kisf = cell_area_kisf * isf_conc_kisf

        T_mean_cav = uf.weighted_mean(T_isf, 'mask_coord', weight_kisf).to_dataset(name='T_mean')
        S_mean_cav = uf.weighted_mean(S_isf, 'mask_coord', weight_kisf).to_dataset(name='S_mean')
        
        T_f_isf = meltf.freezing_temperature(S_isf, -1*depth_kisf)
        TF_isf = T_isf - T_f_isf
        
        TF_mean_cav = uf.weighted_mean(TF_isf, 'mask_coord', weight_kisf).to_dataset(name='TF_mean')
        
        T_S_2D_meanstd_kisf = xr.merge([T_mean_cav,S_mean_cav,TF_mean_cav])
        
        all_info_list.append(T_S_2D_meanstd_kisf.assign_coords({'Nisf': kisf}))

    all_info = xr.concat(all_info_list, dim='Nisf')
    
    outputpath_mean = home_path+'/DATA/SUMMER_PAPER/interim/T_S_mean/'+mod+'/'
    all_info.to_netcdf(outputpath_mean + 'T_S_TF_isfmean_'+scenario+'.nc')